In [ ]:
#-*- coding: utf-8 -*-

## 라이브러리 호출

In [ ]:
# !pip3 install tqdm

In [ ]:
from urllib.request import Request, urlopen
from urllib.parse import unquote, urlencode, quote_plus

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
cwd = os.getcwd()
from tqdm import tqdm

In [ ]:
import datetime

## API KEY

In [ ]:
key = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D'

## 로그

In [ ]:
_nowTime = datetime.datetime.now(tz=datetime.timezone(datetime.timedelta(hours=9)))
nowTime = _nowTime.strftime('%c')
print("[시작] {}".format(nowTime), end='')

In [ ]:
#TODO로그작성
"""
[종료] Wed Jan 28 01:23:08 2021
* (SOLVED)stationNo가 존재 하지 않는 경유 정류소로 인하여 데이터프레임 작성시 데이터프레임이 밀리는 현상이 발생.
    XML을 파싱하는 df_from_xml 함수를 oredereddict로 수정해보자.
"""
"""
[시작] Thu Jan 29 00:16:37 2021
[종료] Thu Jan 29 01:36:31 2021
* (SOLVED)정류장 사이 거리를 구하는 함수를 클래스 메서드로 추가해보자.
"""
"""
[시작] Fri Jan 29 16:44:18 2021
[종료] Fri Jan 29 19:16:17 2021
* Station Class 완성.
* Bus 클래스 메소드 작성: 버스 위치정보를 받아오는 메서드(핵심)를 작성해보자.
"""
pass

## 기본함수

### XML Parser

In [ ]:
def df_from_xml(url, queryParams):
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    raw_xml = urlopen(request).read().decode('utf8')
        
    xtree = ET.fromstring(raw_xml)    
        
    IsColListEmpty = True
    
    col_list = []
    df_list = []
    
    for tags in xtree:
        if(tags.tag == 'msgBody'):
            for busRouteList in tags:
                bus_route = {}
                for col in busRouteList:
                    if(IsColListEmpty):
                        col_list.append(col.tag)
                    bus_route[col.tag] = col.text
                IsColListEmpty = False
                df_list.append(bus_route)

    df = pd.DataFrame(df_list, columns=col_list)
    return df

### GPS상 거리를 구하는 함수

In [ ]:
# !pip3 install haversine
from haversine import haversine
def distance(sx, sy, lx, ly):
    dist = haversine((sy, sx), (ly, lx)) * 1000
    return dist # 기준거리

# RouteId

## 노선번호를  넣으면 노선ID 목록을 반환한다.

In [ ]:
class RouteId:
    def __init__(self, key, routeNo):
        self.routeNo = str(routeNo)
        
        self.key = unquote(key)
        
    def getRouteId(self):
        url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice'
        queryParams = '?' + urlencode({quote_plus('serviceKey') : self.key, quote_plus('keyword') : self.routeNo })
        
        self.df_routeNo = df_from_xml(url, queryParams)
        
        return self.df_routeNo
    
    def routeId(self, index):
        routeId = self.df_routeNo.iloc[index]['routeId']
        return routeId

In [ ]:
_routeNo = str(9003)
route = RouteId(key, _routeNo)
df_busrouteservice = route.getRouteId()
df_busrouteservice

In [ ]:
index = 0
routeNo = df_busrouteservice.iloc[index]['routeName']
routeId = route.routeId(index)

In [ ]:
index, routeNo, routeId

# Station

## Bus Route Service

In [ ]:
class Station:
    def __init__(self, key, routeNo, routeId):
        self.routeNo = str(routeNo)
        self.routeId = str(routeId)
        self.key = unquote(key)
        
    def getStationDf(self):
        url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/station'
        queryParams = '?' + urlencode({quote_plus('serviceKey') : self.key, quote_plus('routeId') : self.routeId })
        
        _df_station = df_from_xml(url, queryParams)
        _df_station.index = _df_station.index + 1
        
        _df_station = _df_station.astype({
            'x' : float,
            'y' : float})
        
        self.df_station = _df_station
        
        return self.df_station
    
    def getLineDf(self):
        url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/line'
        queryParams = '?' + urlencode({quote_plus('serviceKey') : self.key, quote_plus('routeId') : self.routeId })
        
        _df_line = df_from_xml(url, queryParams)
        _df_line.index = _df_line.index+1
        
        _df_line = _df_line.astype({
            'x' : float,
            'y' : float
        })
        
        self.df_line = _df_line.drop(columns='lineSeq')
        
        return self.df_line
    
    def turnYn(self):
        index = df_station['turnYn'] == 'Y'
        self.turnStation = df_station[index]
        turnIndex = df_station[index].index[0]
        return turnIndex
    
#     def stationIndex(self, K=100):
#         df_line = self.df_line
#         df_station = self.df_station
        
#         stationIndexList = []
#         lastIndex = 0
#         K = 100
#         for i in tqdm(df_station.index):

#             distList = []
#             indexList = []

#             for j in range(lastIndex, lastIndex + K):
#                 if(j >= len(df_line)):
#                     break
#                 (sx, sy) = df_station.iloc[i-1][['x', 'y']]
#                 (lx, ly) = df_line.iloc[j][['x', 'y']]
#                 dist = distance(sx, sy, lx, ly)

#                 distList.append(dist)
#                 indexList.append(j)

#             tempIndex = np.argmin(np.array(distList))
#             lastIndex = indexList[tempIndex]

#             stationIndexList.append(lastIndex)
            
#         self.stationIndexList = np.array(stationIndexList, dtype=np.int32)
#         self.df_line_station = df_line.iloc[self.stationIndexList]
            
#         return self.stationIndexList
    
    """
    9003번에 대한 df_line의 staiton_index 추출값.
    추출과정이 오래걸려서 하드코딩.
    클래스 작성이 완료되면 위 메서드를 주석해제후 해당 메서드를 삭제.
    """
    def stationIndex(self, K=100):
        
        df_line = self.df_line
        self.stationIndexList = np.array([  
            0,   6,  12,  19,  27,  34,  41,  47,  53,  60,  65,  86, 107,
           117, 128, 139, 161, 175, 200, 217, 231, 240, 267, 277, 291, 316,
           335, 342, 347, 360, 377, 390, 417, 426, 439, 456, 481, 495, 514,
           528, 541, 550, 572, 596, 602, 608, 613, 620, 628, 635, 643, 650,
           658, 662, 666], dtype=np.int32)
        self.df_line_station = df_line.iloc[self.stationIndexList]
        return self.stationIndexList

    def stationDistance(self):
        df_line = self.df_line
        sumDist = 0
        distList = [0]
        j=1
        for i in tqdm(range(len(df_line)-1)):
            if(i+1 == stationIndexList[j]+1):
                distList.append(sumDist)
                sumDist = 0
                j += 1
            (pre_x, pre_y) = df_line[['x', 'y']].iloc[i]
            (next_x, next_y) = df_line[['x', 'y']].iloc[i+1]
            partDist = distance(pre_x, pre_y, next_x, next_y)
            sumDist += partDist
        distList.append(sumDist)
        
        self.cumsumDist = np.array(distList, dtype=np.int32).cumsum()
        self.distList = np.array(distList, dtype=np.int32)
        
        return self.distList
    
    
    def kakaoMap(self):
        routeNo = self.routeNo
        routeId = self.routeId
        df_station = self.df_station
        
        self.fname_kakaomap_station_location = "{}_{}_kakaomap_staion_location.txt".format(routeNo, routeId)
        f = open(self.fname_kakaomap_station_location, 'w')
        for index in df_station.index:
            stationName, x, y = df_station[['stationName', 'x','y']].iloc[index-1]
            f.write("\t{")
            line = str("content: '<div>{}</div>',latlng: new kakao.maps.LatLng({}, {})").format(stationName, y, x)
            f.write(line)
            f.write("},\n")
            
        f.close()
        print("파일 쓰기 완료")
        
        
    def addColumns(self):
        
        if 'stationIdx' not in self.df_station.columns:
            self.df_station['stationIdx'] = stationIndexList
            print('[stationIdx] 컬럼을 추가하였습니다.')

        if 'distance' not in self.df_station.columns:
            self.df_station['distance'] = distList
            print('[distance] 컬럼을 추가하였습니다.')

        if 'cumsum' not in self.df_station.columns:
            self.df_station['cumsum'] = cumsumDistList    
            print('[cumsum] 컬럼을 추가하였습니다.')

            
    def writeStationDf(self):
        self.fname_station = "{}_{}_station.csv".format(routeNo, routeId)
        self.df_station.to_csv(self.fname_station)
        print("{}_{}_station.csv를 내보냈습니다.".format(routeNo, routeId))

## 인스턴스 생성

### 필수 메서드

In [ ]:
station = Station(key, routeNo, routeId)

df_station = station.getStationDf()
df_line = station.getLineDf()

In [ ]:
stationIndexList = station.stationIndex()

# stationIndexList
distList = station.stationDistance()
# distList

cumsumDistList = station.cumsumDist

station.addColumns()

### 추가 메서드

In [ ]:
station.kakaoMap()
station.writeStationDf()

# Bus

In [ ]:
class Bus:
    def __init__(self, key, routeId):
        self.routeId = str(routeId)
        self.key = unquote(key)
        self.totalBus = None
        self.routeName = None
        
    def getInfoDf(self):
        url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info'
        queryParams = '?' + urlencode({quote_plus('serviceKey') : self.key, quote_plus('routeId') : self.routeId })
        
        _df_info = df_from_xml(url, queryParams)
        _df_info.index = _df_info.index + 1
  
        self.df_info = _df_info
        
        return self.df_info
    
    def basicInfo(self):
        df_info = self.df_info
        self.routeName = df_info.iloc[0]['routeName']
        self.downFirstTime = df_info.iloc[0]['downFirstTime']
        self.upLastTime = df_info.iloc[0]['upLastTime']


In [ ]:
bus = Bus(key, routeId)

In [ ]:
df_info = bus.getInfoDf()

In [ ]:
df_info

In [ ]:
bus.basicInfo()
bus.routeName